In [1]:
!pip install google-search-results

### Importing libraries

In [14]:
from deepface import DeepFace
import requests
from io import BytesIO
from PIL import Image
import json

In [1]:
# Write your SerpAPI key here as api_key:
api_key=""

In [2]:
from serpapi import GoogleSearch

### Input Persona

In [3]:
persona = {
        
#     "name": "Derek Koch",
#     "image": "https://drive.google.com/file/d/1ON-0Ka1UHxedxRX4wI6rvTg-Qlk-Ytx1/view?usp=drive_link",
#     "intro": "The Seeker (https://theseeker.storychamp.community/)",
#     "timezone": "America/Chicago",
#     "company_industry": [],
#     "company_size": [],
#     "social_profile": []
    
    
#     "name": "Manoj Ranaweera",
#     "image": "https://drive.google.com/file/d/1i86Q1Yhi6TpA7rQJXCxE0d_89LJRMLwD/view?usp=drive_link",
#     "intro": "Techcelerate + Deal Lite + SkilledUp Life (https://skilledup.life, https://deallite.uk)",
#     "timezone": "Europe/London",
#     "company_industry": "Tech",
#     "company_size": [],
#     "social_profile": []     
     
    
#     "name": "Gaurav Nemade",
#     "image": "https://drive.google.com/file/d/1IgK6_gHu4lM55WovCqz1rEGuVw0136Kd/view?usp=drive_link",
#     "intro": "Co-founder @ Inventive.ai (RFP + AI) | YC, Founding team @ Google AI, IIT",
#     "timezone": "Asia/Kolkata",
#     "company_industry": "AI",
#     "company_size": [],
#     "social_profile": []
    
#      "name": "Jose Shaji Erinjery",
#      "image": "https://avatars.slack-edge.com/2023-06-08/5418513187520_e314431c7b5c3506ab74_original.jpg",
#      "intro": "Strategy & Commercial Performance",
#      "timezone": "Asia/Kolkata",
#      "company_industry": "Pharmaceuticals",
#      "company_size": [],
#      "social_profile": []
    
    "name": "Vlada Rybak",
    "image": "https://avatars.slack-edge.com/2023-09-25/5942882652514_4ba06621ab980aa47360_original.jpg",
    "intro": [],
    "timezone": "Asia/Jerusalem",
    "company_industry": [],
    "company_size": [],
    "social_profile": []
    
}

### Setting up the search query for Linkedin profiles

In [4]:
def build_search_queries(persona):
    name = persona.get("name", "")
    intro = persona.get("intro", "")

    queries = [
        f'"{name}" site:linkedin.com/in',
        f'"{name}" "{intro}" site:linkedin.com/in',
    ]

    if name and len(name.split()) > 0:
        queries.append(f'"{name.split()[0]}" "{intro}" site:linkedin.com/in')

    return queries

### Function for searching the Linkedin Profiles and extracting data from them, using queries

In [5]:
def search_linkedin_profiles(persona, api_key, max_results=8):
    queries = build_search_queries(persona)
    seen = set()
    candidates = []

    for query in queries:
        params = {
            "engine": "google",
            "q": query,
            "api_key": api_key,
            "num": max_results
        }

        search = GoogleSearch(params)
        results = search.get_dict()

        for result in results.get("organic_results", []):
            link = result.get("link", "")
            if "linkedin.com/in/" in link and link not in seen:
                    candidates.append({
                        "link": link,
                        "title": result.get("title"),
                        "snippet": result.get("snippet"),
                    })
                    seen.add(link)

            if len(candidates) >= max_results:
                break

        if len(candidates) >= max_results:
            break

    return candidates


In [6]:
results = search_linkedin_profiles(persona,api_key,5)

links=[]

for i, r in enumerate(results, 1):
    print(f"{i}. {r['link']}")
    print(f"Title: {r['title']}")
    print(f"Snippet: {r['snippet']}\n")
    links.append(r["link"])

1. https://ua.linkedin.com/in/vlada-rybak-43091b249
Title: Vlada Rybak – Partnership Manager and Outreach specialist
Snippet: Vlada Rybak. Link Building & Outreach Specialist I Open to Link exchange. Jooble. Kyiv, Kyiv City, Ukraine. 4 тис. послідовники 500+ контактів.

2. https://ua.linkedin.com/in/vlada-rybak-83098548
Title: Vlada Rybak – Ukraine | Професійний профіль
Snippet: Регіон: Ukraine. Перегляньте профіль Vlada Rybak на LinkedIn, професійній спільноті з одним мільярдом учасників.

3. https://www.linkedin.com/in/vlada-bolotova-34b911257
Title: Vlada Bolotova - Miami, Florida, United States
Snippet: Vlada Bolotova. University of Miami. Miami, Florida, United States. 587 followers 500+ connections.

4. https://www.linkedin.com/in/vlada-vasilets
Title: Vlada Vasilets - Peackor
Snippet: My name is Vlada, and I'm passionate about helping People Leaders build strong, engaged… · Experience: Peackor · Education: Perm State Humanitarian ...

5. https://ru.linkedin.com/in/vlada-p-93567a

### A list of Linkedin profile urls called Links

In [7]:
links

['https://ua.linkedin.com/in/vlada-rybak-43091b249',
 'https://ua.linkedin.com/in/vlada-rybak-83098548',
 'https://www.linkedin.com/in/vlada-bolotova-34b911257',
 'https://www.linkedin.com/in/vlada-vasilets',
 'https://ru.linkedin.com/in/vlada-p-93567a20a']

### Using BrightData API to fetch results from Linkedin Profile Urls (mainly getting Profile Pic Url)

In [8]:
import requests
import time

def get_profile_brightdata(link, api_key):
    BASE = "https://api.brightdata.com/datasets/v3"
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json",
    }
    
    all_profiles = []

    for link in links:
        link = link.strip() 
    
        params = {
        "dataset_id": "gd_l1viktl72bvl7bjuj0",
        "include_errors": "true",
    }

        trigger = requests.post(
            f"{BASE}/trigger",
            headers=headers,
            params=params,
            json=[{"url": link}],
    )
        trigger.raise_for_status()
        snap_resp = trigger.json()
        snapshot_id = snap_resp.get("snapshot_id")
        if not snapshot_id:
            raise RuntimeError(f"No snapshot_id in trigger response: {snap_resp}")

        print("Triggered snapshot:", snapshot_id)


        download_url = f"{BASE}/snapshot/{snapshot_id}"
        for attempt in range(60):  
            r = requests.get(download_url, headers=headers)

            if r.status_code == 202:
                try:
                    status = r.json().get("status")
                except ValueError:
                    status = "unknown"
                print(f"[{attempt+1:02d}/60] task not completed status={status}")
                time.sleep(5)
                continue

            if r.status_code == 404:
                raise RuntimeError(f"Snapshot not found: {snapshot_id}")

            r.raise_for_status()

            data = r.json()
            print("Snapshot ready, fetched results.")
            
            if "results" in data:
                all_profiles.extend(data["results"])
            else:
                all_profiles.append(data)
                
            break    
            

    return all_profiles

### Note: This code will take some time to run and fetch the profile pic urls.

In [9]:
# Write your BrightData API key:
api_key = ""


profiles = get_profile_brightdata(links, api_key)

filtered_profiles = []
linkedin_urls=[]

for link in links:
    link_results = [p for p in profiles if p.get("url", "").strip("/") == link.strip("/")]
    
    if link_results:
        filtered_profiles.append(link_results[0])
    else:
        filtered_profiles.append({"url": link, "avatar": None, "error": "No exact match found"})

# Step 3: Print avatar URLs

for i, profile in enumerate(filtered_profiles):
    if profile.get("avatar"):
        print(f"\nAvatar #{i+1}: {profile['avatar']}")
        linkedin_urls.append(profile['avatar'])
    else:
        print(f"\nAvatar #{i+1}: [No avatar] – {profile.get('error')}")
        linkedin_urls.append("No avatar")



Triggered snapshot: s_m9kadjpgwfyic3kmd
[01/60] task not completed status=running
[02/60] task not completed status=running
[03/60] task not completed status=running
[04/60] task not completed status=running
[05/60] task not completed status=running
[06/60] task not completed status=running
[07/60] task not completed status=running
[08/60] task not completed status=running
[09/60] task not completed status=running
[10/60] task not completed status=running
[11/60] task not completed status=running
[12/60] task not completed status=running
[13/60] task not completed status=running
[14/60] task not completed status=running
[15/60] task not completed status=running
[16/60] task not completed status=running
[17/60] task not completed status=running
[18/60] task not completed status=running
[19/60] task not completed status=running
[20/60] task not completed status=running
[21/60] task not completed status=running
[22/60] task not completed status=running
[23/60] task not completed status=ru

### A list of Profile pic urls fetched from above code

In [10]:
linkedin_urls

['https://media.licdn.com/dms/image/v2/D4E03AQHcfDBFRewUnw/profile-displayphoto-shrink_200_200/profile-displayphoto-shrink_200_200/0/1695620149191?e=2147483647&v=beta&t=-GE2jsOGQwPs5g8n3RmNnxT2eSMFaRBVQU5Az70uMLQ',
 'https://media.licdn.com/dms/image/v2/C5603AQFUFR1tSqvXwA/profile-displayphoto-shrink_200_200/profile-displayphoto-shrink_200_200/0/1651827932926?e=2147483647&v=beta&t=7GW1MXH_rotjO-N9SIpyw9fkje96ymAJo1qL5-6R4To',
 'https://media.licdn.com/dms/image/v2/D4E03AQF8G42QGC6Buw/profile-displayphoto-shrink_200_200/profile-displayphoto-shrink_200_200/0/1678225840064?e=2147483647&v=beta&t=8hdN9x2vAmUjKrn8vjlTdSDA_b4U2igvxn2JwHfWc6Y',
 'https://media.licdn.com/dms/image/v2/D4E03AQHeknlnxA1gQA/profile-displayphoto-shrink_200_200/B4EZXRndEGGwAY-/0/1742978536503?e=2147483647&v=beta&t=Tq8jIqvvJDzMSyXvD96N-zwWYXoADJKrbACYhhGj3SQ',
 'https://media.licdn.com/dms/image/v2/D4E03AQEQL_2uvobHYg/profile-displayphoto-shrink_200_200/B4EZO_mth_HsAY-/0/1734086417043?e=2147483647&v=beta&t=gcYEcixvNk4

### Getting the input persona image to compare with the linkedin profile pics

In [11]:
reference_url = persona.get("image")

if not reference_url:
    raise ValueError("Reference image URL not found in persona.")

### Function to get the correct path for reference(input persona) image to put it further in DeepFace function for comparison 

In [12]:
def load_reference_image(image_url_or_path):
    # If it's a Google Drive share link
    drive_match = re.search(r'drive\.google\.com\/file\/d\/([^\/]+)', image_url_or_path)
    
    if drive_match:
        file_id = drive_match.group(1)
        direct_url = f"https://drive.google.com/uc?export=download&id={file_id}"
        print(f"Fetching from Google Drive: {direct_url}")
        response = requests.get(direct_url)
        img = Image.open(BytesIO(response.content)).convert("RGB")
#         img = Image.open(BytesIO(response.content))
        img.save("reference.jpg")  # Save temporarily
        return "reference.jpg"     # Return path for DeepFace
    
    elif image_url_or_path.startswith("http"):
        # If it's a direct image URL (.jpg, .png, etc.)
        print(f"Fetching from image URL: {image_url_or_path}")
        response = requests.get(image_url_or_path)
        img = Image.open(BytesIO(response.content))
        img.save("reference.jpg")
        return "reference.jpg"
    
    else:
        # Local file path
        print(f"Using local file: {image_url_or_path}")
        return image_url_or_path


In [15]:
import re
reference_path = load_reference_image(reference_url)

Fetching from image URL: https://avatars.slack-edge.com/2023-09-25/5942882652514_4ba06621ab980aa47360_original.jpg


### Final function to compare both the images and getting the best matched Linkedin profile pic with the score

In [16]:
# Function to download image from URL
def get_image_from_url(url):
    response = requests.get(url)
    response.raise_for_status()
    return Image.open(BytesIO(response.content)).convert("RGB")

# Function to compare reference image against multiple LinkedIn avatars
def compare_reference_to_linkedin(reference_url, linkedin_urls):
    best_match = None
    best_score = float('inf')  # Start with a very high number

    # Loop through LinkedIn image URLs
    for url in linkedin_urls:
        try:
#             img2 = get_image_from_url(url)
            result = DeepFace.verify(img1_path=reference_path, img2_path=url)
            score = result["distance"]

            print(f"LinkedIn URL: {url}")
            print(f"Distance score: {score}")
            print(" ")

            # Track the best match (lowest score)
            if score < best_score:
                best_score = score
                best_match = url
        except Exception as e:
            print(f"Error comparing {url}: {e}")
            print(" ")
    
    return best_match, best_score


# Run the comparison
best_match, best_score = compare_reference_to_linkedin(reference_url, linkedin_urls)

# Output best match
print(" RESULT : ")
if best_match and best_score<0.3:
    print("\nBEST MATCH :")
    print(f"LinkedIn Profile Pic URL: {best_match}")
    print(f"Confidence score: {best_score}")
else:
    print("For image, No match found.")


LinkedIn URL: https://media.licdn.com/dms/image/v2/D4E03AQHcfDBFRewUnw/profile-displayphoto-shrink_200_200/profile-displayphoto-shrink_200_200/0/1695620149191?e=2147483647&v=beta&t=-GE2jsOGQwPs5g8n3RmNnxT2eSMFaRBVQU5Az70uMLQ
Distance score: 0.02115326856648836
 
LinkedIn URL: https://media.licdn.com/dms/image/v2/C5603AQFUFR1tSqvXwA/profile-displayphoto-shrink_200_200/profile-displayphoto-shrink_200_200/0/1651827932926?e=2147483647&v=beta&t=7GW1MXH_rotjO-N9SIpyw9fkje96ymAJo1qL5-6R4To
Distance score: 0.828765092912626
 
LinkedIn URL: https://media.licdn.com/dms/image/v2/D4E03AQF8G42QGC6Buw/profile-displayphoto-shrink_200_200/profile-displayphoto-shrink_200_200/0/1678225840064?e=2147483647&v=beta&t=8hdN9x2vAmUjKrn8vjlTdSDA_b4U2igvxn2JwHfWc6Y
Distance score: 0.8468549956315818
 
LinkedIn URL: https://media.licdn.com/dms/image/v2/D4E03AQHeknlnxA1gQA/profile-displayphoto-shrink_200_200/B4EZXRndEGGwAY-/0/1742978536503?e=2147483647&v=beta&t=Tq8jIqvvJDzMSyXvD96N-zwWYXoADJKrbACYhhGj3SQ
Distance 

### The lower the score, the more confident the model is that the image belong to the same person.